In [ ]:
# Monte Carlo simulation of a Boston Mechanism assignment with partial coordination among top-20.
# Goal: estimate the probability that student rank 21 gets FN (or Kyobo) if they put it as 1st choice,
# under different levels of coordination (c) among the top-20.

import random
import numpy as np
import pandas as pd
from collections import defaultdict, deque

random.seed(42)
np.random.seed(42)

# 1. 기관별 정원을 담은 projects 딕셔너리
projects = {
    "K사": 5,
    "S사": 4,
    "I사": 4,
    "A사": 3,
    "B사": 4,
    "N사": 50,  # '제한없음'을 전체 학생 수보다 큰 값으로 설정
    "T사": 4,
    "Z사": 3,
    "H사": 4,
    "M사": 6,
    "F사": 2,
    "P사": 4
}

# 2. 자동으로 프로젝트 리스트 생성
project_list = list(projects.keys())

# 3. 티어별 기관 이름을 담은 세트(set)
tier1 = {"K사", "S사", "I사"}
tier2 = {"A사", "B사"}
tier3 = {"N사", "T사"} # N사는 제한없음 때문에 티어를 낮춘 것
tier4 = {"Z사", "H사"}
tier5 = {"M사", "F사", "P사"}

In [2]:
N = 42 #총 학생 수

# top_set = list(range(1, 21))  # 상위권 그룹 = 담합 그룹
top_1_10_set = list(range(1, 11))      # NEW: 최상위 그룹 (1~10등)
top_11_20_set = list(range(11, 21))   # NEW: 준상위 그룹 (11~20등)
top_set = top_1_10_set + top_11_20_set # 담합 계산을 위해 전체 상위권 그룹은 유지
rest_set = list(range(21, N+1))

# 티어 우선순위 정의
TIER_ORDER = [tier1, tier2, tier3, tier4, tier5]

# NEW: 최상위 그룹 (1~10등)의 선호 확률
TOP_1_10_PROBS = [0.75, 0.20, 0.04, 0.01, 0.00]

# NEW: 준상위 그룹 (11~20등)의 선호 확률
TOP_11_20_PROBS = [0.45, 0.35, 0.15, 0.03, 0.02]

# 하위권 그룹
REST_SET_PROBS = [0.10, 0.45, 0.35, 0.06, 0.04]

### 함수 정의

In [3]:
# 선호도 설정
def generate_preferences(c_coord: float, rank21_first_choice: str):
    """
    [V3] 상위권 그룹을 세분화하여 학생 선호도를 생성합니다.
    - c_coord: top_set 내에서 담합하는 학생의 비율
    - rank21_first_choice: 분석 대상 학생의 1지망
    """
    prefs = {}
    
    # Helper: 특정 티어 내에서 정원에 비례하여 프로젝트를 랜덤 선택 (기존과 동일)
    def pick_in_tier(tier_set):
        valid_projects = [p for p in project_list if p in tier_set]
        if not valid_projects:
            return random.choice(project_list)
        caps = np.array([projects[p] for p in valid_projects], dtype=float)
        probs = caps / caps.sum()
        return np.random.choice(valid_projects, p=probs)
    
    # 1) 담합 그룹 (Coordinated) - (로직은 기존과 거의 동일)
    k_coord = int(round(c_coord * len(top_set)))
    coord_students = top_set[:k_coord]
    uncoord_students = top_set[k_coord:]

    # Coordinated strategy
    cap_slots = []
    for p, cap in projects.items():
        num_slots = min(cap, N) 
        cap_slots += [p] * num_slots
    
    cap_slots_sorted = []
    for tier in TIER_ORDER:
        cap_slots_sorted += [p for p in cap_slots if p in tier]
    
    coord_first_choices = cap_slots_sorted[:len(coord_students)]
    
    for i, s in enumerate(coord_students):
        first = coord_first_choices[i] if i < len(coord_first_choices) else pick_in_tier(TIER_ORDER[-1])
        remaining = [p for p in project_list if p != first]
        random.shuffle(remaining)
        prefs[s] = [first] + remaining
    
    # 2) 비담합 상위권 그룹 (Uncoordinated Top-set)
    # CHANGED: 이 루프의 내부 로직이 변경되었습니다.
    for s in uncoord_students:
        # NEW: 학생의 등수에 따라 다른 확률분포를 적용합니다.
        if s in top_1_10_set:
            chosen_tier_index = np.random.choice(len(TIER_ORDER), p=TOP_1_10_PROBS)
        else: # s in top_11_20_set
            chosen_tier_index = np.random.choice(len(TIER_ORDER), p=TOP_11_20_PROBS)
            
        chosen_tier = TIER_ORDER[chosen_tier_index]
        first = pick_in_tier(chosen_tier)
        
        remaining = [p for p in project_list if p != first]
        random.shuffle(remaining)
        prefs[s] = [first] + remaining

    # 3) 21등 학생 (분석 대상) - (기존과 동일)
    first = rank21_first_choice
    remaining = [p for p in project_list if p != first]
    random.shuffle(remaining)
    if rest_set:
        prefs[rest_set[0]] = [first] + remaining

    # 4) 그 외 하위권 학생 (Others) - (기존과 동일)
    other_students = rest_set[1:] if rest_set else []
    for s in other_students:
        chosen_tier_index = np.random.choice(len(TIER_ORDER), p=REST_SET_PROBS)
        chosen_tier = TIER_ORDER[chosen_tier_index]
        first = pick_in_tier(chosen_tier)
        
        remaining = [p for p in project_list if p != first]
        random.shuffle(remaining)
        prefs[s] = [first] + remaining

    return prefs

In [4]:
# 보스턴 매커니즘
def boston_mechanism(prefs):
    """
    Execute Boston mechanism for all students 1..N given preference lists.
    Returns assignment dict: student -> project (or None if unassigned).
    """
    capacity_rem = projects.copy()
    assigned = {s: None for s in range(1, N+1)}
    # Build preference queues
    pref_queues = {s: deque(prefs[s]) for s in range(1, N+1)}
    
    # Round-based applications until all projects full or all students exhausted
    while True:
        # Applications this round
        apps = defaultdict(list)
        any_app = False
        for s in range(1, N+1):
            if assigned[s] is None and pref_queues[s]:
                choice = pref_queues[s][0]  # apply to highest remaining
                apps[choice].append(s)
                any_app = True
        
        if not any_app:
            break
        
        # For each project, accept up to remaining capacity by rank order
        for p, group in apps.items():
            if capacity_rem[p] <= 0:
                # All these students will try next option next round
                continue
            # Sort by rank (ascending rank = higher priority)
            group_sorted = sorted(group)
            take = group_sorted[:capacity_rem[p]]
            for s in take:
                assigned[s] = p
            capacity_rem[p] -= len(take)
        
        # Remove chosen project from the queues of all students who applied and didn't get it
        for p, group in apps.items():
            for s in group:
                # If not assigned yet, remove that project from their list
                if assigned[s] is None and pref_queues[s] and pref_queues[s][0] == p:
                    pref_queues[s].popleft()
        # Also for students who got assigned this round, we can clear their queue (not necessary)
    return assigned

In [5]:
def run_trials(num_trials=10000, 
                  c_list=(0.2, 0.5, 0.8), 
                  choice_list=("B사", "A사"),
                  target_rank=21):
    """
    [V2] 개선된 분석 지표로 몬테카를로 시뮬레이션을 실행합니다.
    - target_rank: 분석하고 싶은 학생의 등수
    - choice_list: target_rank 학생이 1지망으로 선택할 기업 목록
    """
    rows = []
    print("시뮬레이션을 시작합니다...")
    
    # --- 실험 설계 ---
    for c in c_list:
        for choice in choice_list:
            # --- 결과 기록 변수 초기화 ---
            got_choice_count = 0
            
            # NEW: 새로운 분석 지표를 위한 변수
            total_tier2_seats_for_top_set = 0
            total_tier2_assignee_ranks = 0
            total_tier2_assignee_count = 0

            # --- 반복 실행 ---
            for i in range(num_trials):
                # V2 선호도 생성 함수 호출
                prefs = generate_preferences(c, choice) 
                assigned = boston_mechanism(prefs)
                
                # --- 결과 기록 ---
                # 1. 목표 학생(21등)이 1지망에 성공했는지 카운트
                if assigned.get(target_rank) == choice:
                    got_choice_count += 1
                
                # NEW: 새로운 지표 계산
                # 2. 2티어에 배정된 학생 리스트 생성
                tier2_assignees = {s: r for s, r in assigned.items() if r in tier2}
                
                if tier2_assignees:
                    # 2-1. 2티어의 좌석을 top_set이 몇 개나 차지했는가?
                    seats_for_top_set = sum(1 for s in tier2_assignees if s in top_set)
                    total_tier2_seats_for_top_set += seats_for_top_set
                    
                    # 2-2. 2티어 배정자들의 평균 등수는 얼마인가?
                    sum_of_ranks = sum(tier2_assignees.keys())
                    num_of_assignees = len(tier2_assignees)
                    total_tier2_assignee_ranks += sum_of_ranks
                    total_tier2_assignee_count += num_of_assignees

            # --- 통계 요약 ---
            # 모든 trial이 끝난 후 평균값 계산
            p_gets_choice = got_choice_count / num_trials
            
            # CHANGED: 새로운 지표들의 평균 계산
            avg_seats_for_top_set = total_tier2_seats_for_top_set / num_trials
            avg_rank_in_tier2 = (total_tier2_assignee_ranks / total_tier2_assignee_count 
                                 if total_tier2_assignee_count > 0 else 0)
            
            rows.append({
                "coordination_level_c": c,
                "my_first_choice": choice,
                "P(gets_my_choice)": p_gets_choice,
                "avg_tier2_seats_for_top_set": avg_seats_for_top_set,
                "avg_rank_in_tier2": avg_rank_in_tier2
            })
            print(f"  - 담합 수준 {c}, 1지망 '{choice}' 시나리오 완료.")
            
    print("시뮬레이션 완료.")
    return pd.DataFrame(rows)

### 실행부

In [6]:
df_results = run_trials(num_trials=8000, 
                           c_list=(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0), # c가 높을 수록 더 많은 인원의 담합
                           choice_list=["B사", "A사"], 
                           target_rank=21)

print(df_results)

시뮬레이션을 시작합니다...
  - 담합 수준 0.0, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.0, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.1, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.1, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.2, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.2, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.3, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.3, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.4, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.4, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.5, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.5, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.6, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.6, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.7, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.7, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.8, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.8, 1지망 'A사' 시나리오 완료.
  - 담합 수준 0.9, 1지망 'B사' 시나리오 완료.
  - 담합 수준 0.9, 1지망 'A사' 시나리오 완료.
  - 담합 수준 1.0, 1지망 'B사' 시나리오 완료.
  - 담합 수준 1.0, 1지망 'A사' 시나리오 완료.
시뮬레이션 완료.
    coordination_level_c my_first_choice  P(gets_my_choice)  \
0                    0.0              B사           0.623125   
1                    0.0              A사           0.572250   
2                    0.1              B사           0.672500

In [7]:
df_results_v2 = run_trials(num_trials=8000, 
                           c_list=(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0), # c가 높을 수록 더 많은 인원의 담합
                           choice_list=["K사", "S사"], 
                           target_rank=21)

print(df_results_v2)

시뮬레이션을 시작합니다...
  - 담합 수준 0.0, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.0, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.1, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.1, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.2, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.2, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.3, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.3, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.4, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.4, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.5, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.5, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.6, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.6, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.7, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.7, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.8, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.8, 1지망 'S사' 시나리오 완료.
  - 담합 수준 0.9, 1지망 'K사' 시나리오 완료.
  - 담합 수준 0.9, 1지망 'S사' 시나리오 완료.
  - 담합 수준 1.0, 1지망 'K사' 시나리오 완료.
  - 담합 수준 1.0, 1지망 'S사' 시나리오 완료.
시뮬레이션 완료.
    coordination_level_c my_first_choice  P(gets_my_choice)  \
0                    0.0              K사           0.492375   
1                    0.0              S사           0.481375   
2                    0.1              K사           0.190000